## Import tools

In [82]:
!pip install -q dtreeviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.3 MB/s eta 0:00:00


In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_gaussian_quantiles
from collections import Counter
from sklearn.ensemble import RandomForestClassifier as SklearnRandomForest
from sklearn.tree import plot_tree, DecisionTreeClassifier as SklearnDecisionTree
from matplotlib.patches import  Rectangle
from matplotlib.colors import ListedColormap
from dtreeviz import decision_boundaries
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Get the data

In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:

df = pd.read_csv("/content/drive/MyDrive/CS114/StudentPerformanceFactors.csv")
df.head(5)

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [93]:
# Kiểm tra các cột trong dataset
print("Các cột trong dataset:", df.columns)

# Giả sử 'Exam_Score' là cột mục tiêu, các cột còn lại là đặc trưng
target_column = 'Exam_Score'  # Thay đổi nếu tên cột khác
feature_columns = [col for col in df.columns if col != target_column]

# Mã hóa các cột phân loại (nếu có)
df_encoded = pd.get_dummies(df, columns=[col for col in feature_columns if df[col].dtype == 'object'], drop_first=True)

# Chia dữ liệu thành features (X) và target (y)
X = df_encoded.drop(columns=[target_column])
y = df_encoded[target_column]


Các cột trong dataset: Index(['Hours_Studied', 'Attendance', 'Parental_Involvement',
       'Access_to_Resources', 'Extracurricular_Activities', 'Sleep_Hours',
       'Previous_Scores', 'Motivation_Level', 'Internet_Access',
       'Tutoring_Sessions', 'Family_Income', 'Teacher_Quality', 'School_Type',
       'Peer_Influence', 'Physical_Activity', 'Learning_Disabilities',
       'Parental_Education_Level', 'Distance_from_Home', 'Gender',
       'Exam_Score'],
      dtype='object')


In [94]:


# Khởi tạo mô hình Decision Tree Regression với min_samples_split = 3 và max_depth = 3
regressor = DecisionTreeRegressor(min_samples_split=5, max_depth=5)

# Huấn luyện mô hình trên tập huấn luyện
regressor.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = regressor.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 6.3079793483322755
R-squared: 0.5537356203615846


## Node class

In [95]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor '''

        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red

        # for leaf node
        self.value = value

## Tree class

In [96]:
class DecisionTreeRegressor1():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''

        # initialize the root of the tree
        self.root = None

        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''

        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["var_red"])

        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''

        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red

        # return best split
        return best_split

    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''

        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction

    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''

        val = np.mean(Y)
        return val

    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)


    def fit(self, X, Y):
        ''' function to train the tree '''
        Y = Y.to_numpy().reshape(-1, 1)
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''

        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

#Change the predict function of the DecisionTreeRegressor Class:
    def predict(self, X):
        ''' function to predict a single data point '''

        X = X.astype(float).values # Convert X to a NumPy array with numeric type
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

## Fit the model

In [97]:
regressor1 = DecisionTreeRegressor1(min_samples_split=5, max_depth=5)
regressor1.fit(X_train,y_train)
regressor1.print_tree()

X_1 <= 82 ? 3.7901995017284307
 left:X_0 <= 20 ? 1.955071946250209
  left:X_1 <= 70 ? 1.4435631701242144
    left:X_0 <= 15 ? 0.8358876227349832
        left:X_0 <= 6 ? 0.5711975020818088
                left:X_1 <= 65 ? 0.7363636363636366
                                left:58.18181818181818
                                right:59.9
                right:X_3 <= 74 ? 0.5768682001419538
                                left:61.4
                                right:62.920529801324506
        right:X_3 <= 74 ? 0.5530312841657352
                left:X_5 <= 0 ? 1.4882849293563547
                                left:80.0
                                right:62.87692307692308
                right:X_8 <= False ? 0.3835767191835222
                                left:64.73958333333333
                                right:63.1
    right:X_0 <= 13 ? 0.8806935635512829
        left:X_0 <= 9 ? 0.4907404866346319
                left:X_4 <= 3 ? 0.5679529110531014
                           

## Test the model

In [104]:
y_pred1 = regressor1.predict(X_test)
mse = mean_squared_error(y_test, y_pred1)
r2 = r2_score(y_test, y_pred1)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 6.325352495648024
R-squared: 0.5525065394814643


In [108]:
# prompt: sử dụng random forest để dự đoán trong dataser trên với 10 tree

# Assuming X and y are already defined as in the provided code.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Initialize and train a Random Forest Regressor with 10 trees
rf_regressor = RandomForestRegressor(n_estimators=1000, random_state=42)
rf_regressor.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_regressor.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest Regression - Mean Squared Error: {mse_rf}")
print(f"Random Forest Regression - R-squared: {r2_rf}")


Random Forest Regression - Mean Squared Error: 4.93962860892587
Random Forest Regression - R-squared: 0.6505409775335942
